# Info

This script is used to analyze fluorescence spectroscopy data.

-----------------------
Created by Jerry LaRue, larue@chapman.edu, 12/2018

Last modified by Jerry LaRue, larue@chapman.edu, 03/2020

# Plot Data

In [1]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output, Javascript
import re
from importlib import reload

import DataTools
reload(DataTools)

##### Defaults #####

data = DataTools.Data()
BackgroundNames = ['None']
Names = ['']

##### Button Functions #####

def UpdateFileList(b) :
    FolderPath.value = FolderPath.value.replace('\\','/')
    FileName.options = data.DataList(FolderPath.value)

def Update_Runs_Clicked(b):
    Runs = data.Runs()
    Runs = [k for k in Runs if Filter.value in k]
    Runs_Selected.options = Runs
    Runs.insert(0,'None')
    Background.options = Runs

def SpectraToClipboard_Clicked(b):
    DataToSave = data.Spectra
    DataToSave.to_clipboard()

def Update_Plot_Clicked(b):
    with out :
        clear_output(True)
        Wavelength = np.zeros((0))
        data.Plot(Runs_Selected.value,Background=Background.value)

def LoadData(b):
    with out :
        clear_output(True)
        
        ##### Load data #####
        
        Data = data.LoadData(FileName.value)
        Runs = data.Runs()
        Runs_Selected.options = Runs
        Runs.insert(0,'None')
        Background.options = Runs

##### Widgets #####

FolderPath = widgets.Text(
    value='../',
    placeholder='Type file path',
    description='Folder',
    layout=Layout(width='80%'),
    style = {'description_width': '150px'},
    disabled=False
)

FileName = widgets.Dropdown(
    options=data.DataList(FolderPath.value),
    description='File',
    layout=Layout(width='80%'),
    style = {'description_width': '150px'},
    disabled=False,
)

button_UpdateFileList = widgets.Button(description="Update")
button_UpdateFileList.on_click(UpdateFileList)

button_LoadData = widgets.Button(description="Load data")
button_LoadData.on_click(LoadData)

Filter = widgets.Text(
    value='',
    placeholder='Type something',
    description='Filter:',
    style = {'description_width': '150px'},
    disabled=False
)

Update_Runs = widgets.Button(description="Update run list")
Update_Runs.on_click(Update_Runs_Clicked)

SpectraToClipboard = widgets.Button(description="Copy plot data")
SpectraToClipboard.on_click(SpectraToClipboard_Clicked)

Background = widgets.Dropdown(
    options=BackgroundNames,
    value='None',
    layout=Layout(width='80%'),
    description='Select background run',
    style = {'description_width': '150px'},
    disabled=False,
)

Runs_Selected = widgets.SelectMultiple(
    options=Names,
    style = {'width': '100px','description_width': '150px'},
    rows=20,
    layout=Layout(width='80%'),
    description='Select runs',
    disabled=False
)

Update_Plot = widgets.Button(description="Plot")
Update_Plot.on_click(Update_Plot_Clicked)

display(widgets.Box([FolderPath,button_UpdateFileList]))
display(widgets.Box([FileName,button_LoadData]))
display(widgets.Box([Filter,Update_Runs]))
display(Runs_Selected)
display(Background)
display(widgets.Box([Update_Plot,SpectraToClipboard]))

out = widgets.Output()
out

Box(children=(Text(value='../', description='Folder', layout=Layout(width='80%'), placeholder='Type file path'…

Box(children=(Dropdown(description='File', layout=Layout(width='80%'), options=('Buffer Calculations.xlsx',), …

Box(children=(Text(value='', description='Filter:', placeholder='Type something', style=DescriptionStyle(descr…

SelectMultiple(description='Select runs', layout=Layout(width='80%'), options=('',), rows=20, style=Descriptio…

Dropdown(description='Select background run', layout=Layout(width='80%'), options=('None',), style=Description…

Box(children=(Button(description='Plot', style=ButtonStyle()), Button(description='Copy plot data', style=Butt…

Output()

***
# Analyze Data

In [41]:
import lmfit
from lmfit import model
from lmfit.models import SkewedGaussianModel, GaussianModel, LinearModel, VoigtModel, PolynomialModel

import DataTools
reload(DataTools)

##### Button Functions #####

def OkMethod(b) :
    with out2 :
        clear_output(True)
        
        def ResultsToClipboard_Clicked(b):
            AnalysisResults.to_clipboard()
        
        if Analysis.value == 'Fit Data' :

            ###### Setup arrays #####

            B_Slope = list()
            B_Intercept = list()
            G_Amplitude = list()
            G_Center = list()
            G_Sigma = list()
            G_Gamma = list()
            Wavelength = list()
            Amplitude_Norm = list()

            ##### Determine reference fit function #####

            Model = LinearModel(prefix='B_')
            Model = Model + SkewedGaussianModel(prefix='G_')
            Params = Model.make_params()

            ##### Fit data #####

            i = 0
            while i < len(Runs_Selected.value) :
                x = data.Spectra.iloc[:,0]
                y = data.Spectra.iloc[:,i+1]
                
                if MaxValue.value == True :
                    CenterWavelength = x[y.idxmax()]
                else :
                    CenterWavelength = SetWavelength.value
                
                ##### Define constraints #####

                Params['B_slope'].value = 0
                Params['B_slope'].vary = False
                Params['B_intercept'].value = 0
                Params['B_intercept'].vary = False
                Params['G_amplitude'].value = y.max() * 50
                Params['G_amplitude'].min = 0
                Params['G_center'].value = CenterWavelength
                Params['G_sigma'].value = SetWidth.value
                Params['G_sigma'].max = 100
                Params['G_gamma'].value = 3.5
                
                ##### Fit the data #####
                
                Fit = Model.fit(y, Params, x=x, fit_kws={'maxfev': 1000})
                Comps = Fit.eval_components(Fit.params, x=x)

                ##### Plot reference data and fits #####

                plt.plot(x, y,'r.', label='data')
                plt.plot(x, Fit.best_fit, 'k-', label='fit')
                plt.plot(x, Comps['B_'], 'k--', label='Baseline')
                plt.plot(x, Comps['G_'], 'b--', label='Gaussian')
                plt.legend(), plt.xlabel('Wavelength (nm)'), plt.ylabel('Intensity (au)')
                plt.title('Fluorescence')
                plt.show()
                print('Skewed Gaussian | Amplitude: ' + str(round(Fit.params['G_amplitude'].value,4)) + ', Center: ' + str(round(Fit.params['G_center'].value,1)) + ', Width: ' + str(round(Fit.params['G_sigma'].value,3)) + ', Skew: ' + str(round(Fit.params['G_gamma'].value,3)))
                B_Slope.append(Fit.params['B_slope'].value)
                B_Intercept.append(Fit.params['B_intercept'].value)
                G_Amplitude.append(Fit.params['G_amplitude'].value)
                G_Center.append(Fit.params['G_center'].value)
                G_Sigma.append(Fit.params['G_sigma'].value)
                G_Gamma.append(Fit.params['G_gamma'].value)
                Wavelength.append(x[np.argmax(Fit.best_fit)])
                Amplitude_Norm.append(G_Amplitude[i]/G_Amplitude[0])
                i+=1

                print('_____________________________________________________________________________________')

            plt.figure(figsize=(10,5))
            plt.xlabel('Run'), plt.ylabel('Amplitude')
            plt.plot(G_Amplitude,'.-')
            plt.show()

            plt.figure(figsize=(10,5))
            plt.xlabel('Run'), plt.ylabel('Wavelength')
            plt.plot(Wavelength,'.-')
            plt.show()
            
            DataHeader = list(('Wavelength','Amplitude (Max)','Amplitude (Norm)'))
            AnalysisResults = np.zeros((3,len(Wavelength)))
            AnalysisResults[0] = Wavelength
            AnalysisResults[1] = G_Amplitude
            AnalysisResults[2] = Amplitude_Norm
            AnalysisResults = pd.DataFrame(data=np.transpose(AnalysisResults),columns=DataHeader)
            AnalysisResults['Name'] = Runs_Selected.value
            
        if Analysis.value == 'Numerically Integrate' :
            
            Integrated = list()
            Wavelength = list()
            i = 0
            while i < len(Runs_Selected.value) :
                x = data.Spectra.iloc[:,0]
                y = data.Spectra.iloc[:,i+1]
                
                if MaxValue.value == True :
                    CenterIndex = y.idxmax()
                else :
                    CenterIndex = x[x == SetWavelength.value].index[0]
                    
                Index = list(range(int(CenterIndex-SetWidth.value/2),int(CenterIndex+SetWidth.value/2+1)))
                Integrated.append(np.sum(y[Index]) * (x[1]-x[0]))
                Wavelength.append(x[CenterIndex])
                i+=1
            
            AnalysisResults = pd.DataFrame(data=np.transpose((Wavelength,Integrated)),columns=('Wavelength','Area'))
            AnalysisResults['Name'] = Runs_Selected.value
            
            plt.figure(figsize=(10,5))
            plt.xlabel('Run'), plt.ylabel('Intensity')
            plt.plot(Integrated,'.-')
            plt.show()

            plt.figure(figsize=(10,5))
            plt.xlabel('Run'), plt.ylabel('Wavelength')
            plt.plot(Wavelength,'.-')
            plt.show()
            
        ResultsToClipboard = widgets.Button(description="Copy results")
        ResultsToClipboard.on_click(ResultsToClipboard_Clicked)

        display(ResultsToClipboard)

##### Widgets #####

Analysis = widgets.Dropdown(
    options=['Numerically Integrate','Fit Data'],
    description='Analysis Method',
    layout=Layout(width='35%'),
    style = {'description_width': '150px'},
    disabled=False,
)

SetWavelength = widgets.BoundedFloatText(
    value=int(np.mean(data.Spectra.iloc[:,0])),
    min=1,
    max=1000,
    step=1,
    description='Wavelength (nm)',
    layout=Layout(width='22%'),
    style = {'description_width': '150px'},
    disabled=False
)

SetWidth = widgets.BoundedFloatText(
    value=int(40),
    min=1,
    max=200,
    step=1,
    description='Width (nm)',
    layout=Layout(width='22%'),
    style = {'description_width': '150px'},
    disabled=False
)

MaxValue = widgets.Checkbox(
    value=False,
    description='Use most intense data point to determine wavelength',
    layout=Layout(width='60%'),
    style = {'description_width': '150px'},
    disabled=False
)

button_OkMethod = widgets.Button(description="OK")
button_OkMethod.on_click(OkMethod)

display(widgets.Box([Analysis,button_OkMethod]))
display(widgets.Box([SetWavelength,SetWidth]))
display(MaxValue)

out2 = widgets.Output()
out2

Box(children=(Dropdown(description='Analysis Method', layout=Layout(width='35%'), options=('Numerically Integr…

Box(children=(BoundedFloatText(value=380.0, description='Wavelength (nm)', layout=Layout(width='22%'), max=100…

Checkbox(value=False, description='Use most intense data point to determine wavelength', layout=Layout(width='…

Output()